In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import json

In [3]:
!pip install scrapy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.4/286.4 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.1/247.1 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.7/97.7 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 9.8 MB/s eta 0:00:00


In [ ]:
#!scrapy startproject tutorial

In [ ]:
#!scrapy crawl quotes

In [4]:
%cd /content/drive/MyDrive/CS114/tutorial

/content/drive/MyDrive/CS114/tutorial


In [ ]:
import scrapy

class NewsSpider(scrapy.Spider):
    name = "news"

    def start_requests(self):
        urls = [
            'https://vnexpress.net/tag/kinh-te-kho-khan-123023-ap1'
        ]
        for url in urls:
            yield scrapy.Request(url, callback=self.parse)

    def parse(self, response):
        allLinks = response.css('h3.title-news a::attr(href)').getall()

        for each_link in allLinks:
            yield scrapy.Request(each_link, callback=self.parse_href)

        next_page = response.css('a.btn-page next-page::attr(href)').get()
        if next_page is not None:
            next_page = response.urljoin(next_page)
            yield scrapy.Request(next_page, callback=self.parse)

    def parse_href(self, response):
        title = response.css('.title-detail::text').get()
        title = title.replace('\n', '')
        title = title.replace('\t', '')
        paragraph = response.css('.fck_detail p::text').getall()
        text = ''
        count = 0
        for sentence in paragraph:
            if count == 4:
                break
            sentence = sentence.replace('\xa0', '')
            text = text + sentence
            count += 1
        yield {
            'title': title,
            'text': text,
        }


In [8]:
!scrapy crawl news -o benh.json

2024-01-27 02:19:22 [scrapy.utils.log] INFO: Scrapy 2.11.0 started (bot: tutorial)
2024-01-27 02:19:22 [scrapy.utils.log] INFO: Versions: lxml 4.9.4.0, libxml2 2.10.3, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.2, Twisted 22.10.0, Python 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0], pyOpenSSL 24.0.0 (OpenSSL 3.2.0 23 Nov 2023), cryptography 42.0.0, Platform Linux-6.1.58+-x86_64-with-glibc2.35
2024-01-27 02:19:22 [scrapy.addons] INFO: Enabled addons:
[]
2024-01-27 02:19:22 [asyncio] DEBUG: Using selector: EpollSelector
2024-01-27 02:19:22 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.asyncioreactor.AsyncioSelectorReactor
2024-01-27 02:19:22 [scrapy.utils.log] DEBUG: Using asyncio event loop: asyncio.unix_events._UnixSelectorEventLoop
2024-01-27 02:19:22 [scrapy.extensions.telnet] INFO: Telnet Password: c7a3db7cdd41c848
2024-01-27 02:19:22 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 's

In [9]:
data = pd.read_json('/content/drive/MyDrive/CS114/tutorial/benh.json')
data

,title,text
0,Từ chối chữa ung thư vì sợ con cháu ra đường ở,"Một hôm, ông bác hàng xóm kéo tôi qua nhà để t..."
1,Ăn thế nào để lẩy lùi bệnh tật,Một chế độ ăn uống lành mạnh bao gồm thực phẩm...
2,Những người mới U50 đã rệu rã,"Tuần rồi, ông bạn mới 46 tuổi của tôi xin nghỉ..."
3,Mua bảo hiểm nhân thọ cho trụ cột gia đình,Trước đây tôi có hơi kỳ thị về bảo hiểm nhân t...
4,Những sự thật về tuổi già,"Tuổi già là sự khởi đầu một thời kỳ ""vàng son""..."
...,...,...
227,Sai lầm khi dùng kháng sinh nhiều người mắc phải,\nTheo
228,Lợi ích khi ngủ đủ 8 giờ mỗi ngày,\nBạn ngủ bao nhiêu tiếng mỗi đêm? Mỗi người s...
229,"Ngủ quá nhiều, ho dai dẳng - dấu hiệu cần kiểm...","\nTheo , các nhà nghiên cứu cho biết, nhiều bệ..."
230,25% người Việt bị rối loạn tâm thần,\nTheo


In [10]:
import pandas as pd
import json

# Đọc dữ liệu từ file JSON
with open('benh.json', 'r') as json_file:
    data = json.load(json_file)

# Chuyển đổi thành DataFrame
df = pd.json_normalize(data)

# Lưu DataFrame thành file CSV
df.to_csv('benh.csv', index=False)

print("Chuyển đổi thành công. File CSV đã được lưu.")


Chuyển đổi thành công. File CSV đã được lưu.
